<a href="https://colab.research.google.com/github/aneeshcheriank/Advanced-vision/blob/main/impacat_of_Data_Augmentation_and_Batch_Normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Conditions
- No Batch normalization & No Data Augmentation
- Batch normalization & No Data Augmentation
- Batch normalization & Data Augmenation

In [ ]:
# required packages
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models, datasets

!pip install torch_summary
from torchsummary import summary
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

import matplotlib.pyplot as plt
from PIL import Image # what is the use of PIL.image?
import cv2, glob, numpy as np, pandas as pd
from glob import glob
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [ ]:
# download the dataset
!pip install -q kaggle
from google.colab import files
files.upload()
# upload the kaggle authentication token 
# to download the data
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# download and unzip the file
!kaggle competitions download -c dogs-vs-cats
!unzip dogs-vs-cats.zip
!unzip -q train.zip -d data
!unzip -q test1.zip -d data

In [ ]:
# Define transforms on the data
from torchvision import transforms as T
trn_tfms = T.Compose([
  T.ToPILImage(),
  T.Resize(224),
  T.CenterCrop(224),
  T.ColorJitter(brightness=(0.8, 1.2),
    contrast=(0.8, 1.2),
    saturation=(0.8, 1.2),
    hue=0.25
  ),
  T.RandomAffine(5, translate=(0.01, 0.1)),
  T.ToTensor(),
  T.Normalize(mean=(.45, .44, .43),
              std=(0.22, 0.23, 0.24)
              )
  ])

val_tfms = T.Compose([
  T.ToPILImage(),
  T.Resize(32),
  T.CenterCrop(32),
  T.ToTensor(),
  T.Normalize(mean=(.45, .44, .43),
              std=(0.22, 0.23, 0.24)
              )
  ])

In [ ]:
train_data_dir = 'data/train'
test_data_dir = 'data/train'

class CatsDogs(Dataset):
  def __init__(self, folder, transform=None):
    cats = glob(folder+'/cat.*.jpg')
    dogs = glob(folder+'/dog.*.jpg')
    self.fpaths = cats + dogs
    
    self.transform = transform
    from random import shuffle, seed
    seed(10)
    shuffle(self.fpaths)
    # target: name with cat is True else False
    # print(self.fpaths[0])
    self.targets = [fpath.split('/')[2].split('.')[0] == 'cat'\
      for fpath in self.fpaths]
  
  def __getitem__(self, ix):
    f = self.fpaths[ix]
    target = int(self.targets[ix])
    # convert the targets to tensor
    target = torch.tensor(target)
    im = (cv2.imread(f)) # need to change the image from BGR to RBG
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

    return im, target

  def __len__(self):
    return len(self.fpaths)

  def collate_fn(self, batch):
    imgs, cls = list(zip(*batch))
    cls = torch.tensor(cls).float()
    if self.transform:
      imgs = [self.transform(img)[None, :, :, :] for img in imgs]
    imgs = torch.cat(imgs)
    imgs = imgs.float()
    return imgs, cls


# fetch the images and their labels
data = CatsDogs(train_data_dir)

im, label = data[200]
plt.imshow(im)
print(label)

In [ ]:
train_dt = CatsDogs(train_data_dir, trn_tfms)
train_dl = DataLoader(
  train_dt, batch_size = 128, shuffle=True, 
  collate_fn = train_dt.collate_fn, drop_last=True
)

# test data loader
val_dt = CatsDogs(train_data_dir, val_tfms)
val_dl = DataLoader(
  val_dt, batch_size = 128, shuffle=False, collate_fn = train_dt.collate_fn, 
  drop_last=True
)

# to test the data loader
imgs, labels = next(iter(train_dl))
plt.figure(figsize=(4, 4))
plt.imshow(imgs[0].permute(2, 1, 0))
plt.axis(False)
plt.show()
# will through an error, since the normalized image +ve and -ve laues

In [ ]:
def convBlock(ni, no):
  return nn.Sequential(
      nn.Dropout(0.2),
      nn.Conv2d(ni, no, 3, padding = 1),
      nn.ReLU(inplace=True),
      nn.BatchNorm2d(no),
      nn.MaxPool2d(2)
  )

class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()
    self.model = nn.Sequential(
        convBlock(3, 64),
        convBlock(64, 64),
        convBlock(64, 128),
        convBlock(128, 256),
        nn.Flatten(),
        nn.Linear(14*14*256, 256),
        nn.Dropout(0.2),
        nn.ReLU(inplace=True),
        nn.Linear(256, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    x = self.model(x)
    return x

model = Classifier()
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# define a fucntion to train and evaluate the model on a batch
def train_batch(data, model, optimizer, criterion):
  model.train()
  ims, labels = data
  ims = ims.to(device)
  labels = labels.to(device)
  pred = model(ims)
  loss = criterion(pred, labels[:, None])
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  # calculate accuracy
  pred_label = torch.argmax(pred, -1)
  acc = ((pred_label > 0.5) == label).sum()/len(pred_label)

  return loss.item(), acc.item()

@torch.no_grad()
def validate_batch(data, model, criterion):
  model.eval()
  ims, label = data
  ims = ims.to(device)
  label = label.to(device)
  pred = model(ims)
  loss = criterion(pred, label[:, None])
  # calculate accuracy
  pred_label = torch.argmax(pred, -1)
  acc = ((pred_label > 0.5) == label).sum()/len(pred_label)

  return loss.item(), acc.item()

In [ ]:
# training loop
model = Classifier().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
n_epochs = 50

train_losses, train_accs, val_losses, val_accs = [], [], [], []
for epoch in tqdm(range(n_epochs)):
  print(f'epoch: {epoch+1}/{n_epochs}')
  batch_loss, batch_acc = [], []
  for batch, data in enumerate(train_dl):
    loss, acc = train_batch(data, model, optimizer, loss_fn)
    # append the batch metrics
    batch_loss.append(loss)
    batch_acc.append(acc)
  
  batch_val_loss, batch_val_acc = [], []
  for batch, data in enumerate(val_dl):
    loss, acc = validate_batch(data, model, loss_fn)
    # append the validation batch metrics
    batch_val_loss.append(loss)
    batch_val_acc.append(acc)
  
  train_loss = np.mean(batch_loss)
  train_acc = np.mean(batch_acc)
  val_loss = np.mean(batch_val_loss)
  val_acc = np.mean(batch_val_acc)

  # append the metics
  train_losses.append(train_loss)
  train_accs.append(train_accs)
  val_losses.append(val_losses)
  val_accs.append(val_acc)

  print(f'train loss: {train_loss}; train acc: {train_acc}; validation loss: {val_loss}; validation acc: {val_acc}')